### 라이브러리 및 데이터 불러오기


In [1]:
# 기본 라이브러리
import numpy as np
import pandas as pd

# 시각화 라이브러리
import matplotlib.pyplot as plt 
import seaborn as sns
sns.set_style('darkgrid')

# 모델링을 위한 sklearn 패키지
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score, KFold #***
from sklearn.linear_model import RidgeCV, Ridge, LassoCV, Lasso, ElasticNetCV, ElasticNet #***
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor,VotingRegressor #***^
from sklearn.linear_model import LinearRegression

# score를 내줄 함수
from sklearn import metrics
from sklearn.metrics import make_scorer
from sklearn.metrics import mean_squared_log_error

# 모델링에 활용한 패키지
import lightgbm as lgbm 
from lightgbm import LGBMRegressor

import xgboost as xgb 
from xgboost import XGBRegressor #XGBRegressor

import warnings
# warnings.filterwarnings('ignore')

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
train = pd.read_csv('/content/drive/MyDrive/Colab Notebooks (1)/content/preprocessed_train_2021-12-07.csv')
train.head()

,datetime,holiday,workingday,atemp,humidity,windspeed,casual,registered,count,day,rainyday,ideal,sticky,peak,hour_0,hour_1,hour_2,hour_3,hour_5,hour_6,hour_7,hour_8,hour_9,hour_10,hour_11,hour_12,hour_13,hour_14,hour_15,hour_16,hour_17,hour_18,hour_19,hour_20,hour_21,hour_22,hour_23,hw_0,hw_1,hw_2,...,hw_8,hw_9,hw_10,hw_11,hw_12,hw_13,hw_14,hw_15,hw_16,hw_17,hw_18,hw_19,hw_20,hw_21,hw_22,hw_23,year_2012,dayofweek_0,dayofweek_2,dayofweek_3,dayofweek_4,dayofweek_5,dayofweek_6,season_2,season_3,season_4,month_2,month_3,month_4,month_5,month_6,month_7,month_8,month_9,month_10,month_11,month_12,log_count,temp(difference),discomfort_index
0,2011-01-01 00:00:00,0,0,0.305068,0.81,0.219409,3,13,16,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2.833213,0.111895,0.221012
1,2011-01-01 01:00:00,0,0,0.288064,0.80,0.219409,8,32,40,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3.713572,0.109971,0.193893
2,2011-01-01 02:00:00,0,0,0.288064,0.80,0.219409,5,27,32,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3.496508,0.109971,0.193893
3,2011-01-01 03:00:00,0,0,0.305068,0.75,0.219409,3,10,13,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2.639057,0.111895,0.226778
4,2011-01-01 04:00:00,0,0,0.305068,0.75,0.219409,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.693147,0.111895,0.226778


In [4]:
test = pd.read_csv('/content/drive/MyDrive/Colab Notebooks (1)/content/preprocessed_test_2021-12-02.csv')
test.tail()

,datetime,holiday,workingday,atemp,humidity,windspeed,day,rainyday,ideal,sticky,peak,hour_0,hour_1,hour_2,hour_3,hour_5,hour_6,hour_7,hour_8,hour_9,hour_10,hour_11,hour_12,hour_13,hour_14,hour_15,hour_16,hour_17,hour_18,hour_19,hour_20,hour_21,hour_22,hour_23,hw_0,hw_1,hw_2,hw_3,hw_5,hw_6,hw_7,hw_8,hw_9,hw_10,hw_11,hw_12,hw_13,hw_14,hw_15,hw_16,hw_17,hw_18,hw_19,hw_20,hw_21,hw_22,hw_23,year_2012,dayofweek_0,dayofweek_2,dayofweek_3,dayofweek_4,dayofweek_5,dayofweek_6,season_2,season_3,season_4,month_2,month_3,month_4,month_5,month_6,month_7,month_8,month_9,month_10,month_11,month_12,temp(difference),discomfort_index
6488,2012-12-31 19:00:00,0,1,0.2576,0.523810,0.099973,31,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0.696541,0.274018
6489,2012-12-31 20:00:00,0,1,0.2576,0.523810,0.099973,31,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0.696541,0.274018
6490,2012-12-31 21:00:00,0,1,0.2576,0.523810,0.099973,31,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0.696541,0.274018
6491,2012-12-31 22:00:00,0,1,0.2727,0.476190,0.059904,31,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0.637579,0.276935
6492,2012-12-31 23:00:00,0,1,0.2727,0.583333,0.059904,31,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0.637579,0.270323


### 베이스라인 모델링
특정 기법을 통해 학습 및 평가했을 때, 기존의 모델보다 좋아졌는지 판단하기 위해 기준으로 삼을 베이스라인 모델을 생성한다.

In [5]:
# 전처리한 변수들이 있는데 이 중에서 분석에 활용한 변수를 선택해줍시다.
# 기존 데이터에 덮어쓰기보단 train_copy라는 예비 데이터프레임을 생성하여 저장해줍시다.
train_copy = train

col = ['holiday', 'workingday', 'atemp', 'humidity', 'windspeed',
       'rainyday', 'ideal', 'sticky', 'peak', 'temp(difference)',
       'discomfort_index', 'hour_0', 'hour_1', 'hour_2', 'hour_3', 'hour_5',
       'hour_6', 'hour_7', 'hour_8', 'hour_9', 'hour_10', 'hour_11', 'hour_12',
       'hour_13', 'hour_14', 'hour_15', 'hour_16', 'hour_17', 'hour_18',
       'hour_19', 'hour_20', 'hour_21', 'hour_22', 'hour_23', 'hw_0', 'hw_1',
       'hw_2', 'hw_3', 'hw_5', 'hw_6', 'hw_7', 'hw_8', 'hw_9', 'hw_10',
       'hw_11', 'hw_12', 'hw_13', 'hw_14', 'hw_15', 'hw_16', 'hw_17', 'hw_18',
       'hw_19', 'hw_20', 'hw_21', 'hw_22', 'hw_23', 'year_2012', 'dayofweek_0',
       'dayofweek_2', 'dayofweek_3', 'dayofweek_4', 'dayofweek_5',
       'dayofweek_6', 'season_2', 'season_3', 'season_4', 'month_2', 'month_3',
       'month_4', 'month_5', 'month_6', 'month_7', 'month_8', 'month_9',
       'month_10', 'month_11', 'month_12']

# count를 제외한 변수들을 담은 데이터프레임.
X_features = train[col]
X_test = test[col]

# 타겟 변수는 log 처리를 해준 count 변수
target = train['log_count']

# 데이터를 나눠줌
X_train, X_valid, y_train, y_valid = train_test_split(X_features, target, test_size = 0.3, random_state = 0)

In [6]:
train.columns

Index(['datetime', 'holiday', 'workingday', 'atemp', 'humidity', 'windspeed',
       'casual', 'registered', 'count', 'day', 'rainyday', 'ideal', 'sticky',
       'peak', 'hour_0', 'hour_1', 'hour_2', 'hour_3', 'hour_5', 'hour_6',
       'hour_7', 'hour_8', 'hour_9', 'hour_10', 'hour_11', 'hour_12',
       'hour_13', 'hour_14', 'hour_15', 'hour_16', 'hour_17', 'hour_18',
       'hour_19', 'hour_20', 'hour_21', 'hour_22', 'hour_23', 'hw_0', 'hw_1',
       'hw_2', 'hw_3', 'hw_5', 'hw_6', 'hw_7', 'hw_8', 'hw_9', 'hw_10',
       'hw_11', 'hw_12', 'hw_13', 'hw_14', 'hw_15', 'hw_16', 'hw_17', 'hw_18',
       'hw_19', 'hw_20', 'hw_21', 'hw_22', 'hw_23', 'year_2012', 'dayofweek_0',
       'dayofweek_2', 'dayofweek_3', 'dayofweek_4', 'dayofweek_5',
       'dayofweek_6', 'season_2', 'season_3', 'season_4', 'month_2', 'month_3',
       'month_4', 'month_5', 'month_6', 'month_7', 'month_8', 'month_9',
       'month_10', 'month_11', 'month_12', 'log_count', 'temp(difference)',
       'discomfort_in

In [7]:
# RMSLE 값을 출력하는 함수
def rmsle(y,y_,convertExp=True):
    # 지수화 필요하다면
    if convertExp:
        y = np.exp(y),
        y_ = np.exp(y_)
    log1 = np.nan_to_num(np.array([np.log(v + 1) for v in y]))
    log2 = np.nan_to_num(np.array([np.log(v + 1) for v in y_]))
    calc = (log1 - log2) ** 2
    return np.sqrt(np.mean(calc))

rmsle_scorer = make_scorer(rmsle)
rmsle_scorer

make_scorer(rmsle)

In [8]:
## cross val score를 측정해주는 함수
def cv_score(models, n = 5):
    # kfold 수는 default인 5로 지정
    kf = KFold(n_splits = n, shuffle=True, random_state = 0)
    
    for model in models:
#       model.fit(X_train,y_train)
        score =  cross_val_score(model, X_features, target, cv = kf, scoring=rmsle_scorer)
        print(model[0],'의 평균 score:', round(score.mean(), 5))
        print(model[0],'의 std:', round(score.std(), 5))
        print()      
        
        # y_valid과 prediction을 비교하여 시각화 해주는 코드
#        g = sns.kdeplot(np.exp(y_valid),  color = 'skyblue', alpha = .6, fill = True, label = 'valid')
#        g = sns.kdeplot(np.exp(model.predict(X_valid)), color = 'orange', alpha = .3, fill = True, label = 'prediction')
#       plt.legend()
#        plt.show()

In [9]:
## 제출을 위한 함수
def submission(model):
    model.fit(X_features, target)
    prediction = np.exp(model.predict(X_test))
    
    # 자동으로 형식을 맞춰 csv 생성해주는 코드
    submission = pd.DataFrame(test['datetime'])
    submission['count'] = prediction

    pd.DataFrame(submission).to_csv('submission_bike.csv', index = False)
    
    return pd.DataFrame(submission)

In [10]:
from sklearn.linear_model import ElasticNet
# 기본 모델을 아래와 같이 5가지로 정했음.                                             mean val_score    std
pipe_lr = Pipeline([('model', LinearRegression())])                                 #   0.33503     / 0.00783
pipe_rf = Pipeline([('model', RandomForestRegressor(n_estimators=500))])            #   0.4394      / 0.01136 
pipe_lgbm = Pipeline([('model', LGBMRegressor(n_estimators=100))])                  #   0.32172     / 0.0055
pipe_gb = Pipeline([('model', GradientBoostingRegressor())])                        #   0.59616     / 0.01654
pipe_xgb = Pipeline([('model', XGBRegressor(objective ='reg:squarederror'))])       #   0.59705     / 0.0167

models = [pipe_lr, pipe_rf, pipe_lgbm, pipe_gb, pipe_xgb]

# 평균 valid score 측정
cv_score(models)

LinearRegression() 의 평균 score: 0.33529
LinearRegression() 의 std: 0.00761

RandomForestRegressor(n_estimators=500) 의 평균 score: 0.43904
RandomForestRegressor(n_estimators=500) 의 std: 0.01188

LGBMRegressor() 의 평균 score: 0.32172
LGBMRegressor() 의 std: 0.0055

GradientBoostingRegressor() 의 평균 score: 0.59613
GradientBoostingRegressor() 의 std: 0.01656

XGBRegressor(objective='reg:squarederror') 의 평균 score: 0.59705
XGBRegressor(objective='reg:squarederror') 의 std: 0.0167



In [11]:
# 보팅앙상블^
pipe_voting = Pipeline([('model', VotingRegressor(estimators=[('lr', LinearRegression()), ('rf', RandomForestRegressor(n_estimators=500))]))])   # mean val_score   std
models = [pipe_voting]                                                                                                                           #   0.33762      0.00791

# 평균 valid score 측정
cv_score(models)

VotingRegressor(estimators=[('lr', LinearRegression()),
                            ('rf', RandomForestRegressor(n_estimators=500))]) 의 평균 score: 0.33762
VotingRegressor(estimators=[('lr', LinearRegression()),
                            ('rf', RandomForestRegressor(n_estimators=500))]) 의 std: 0.00791



In [12]:
# lr 모델 제출 결과 : => inf 값 나와서 실패.. 왜지...
# rf 모델 제출 결과 : 0.52784  **
# lgbm 모델 제출 결과 : 0.41477  **
# gb 모델 제출 결과 : 0.66858
# xgb 모델 제출 결과 : 0.52387  **
# voting 모델 제출 결과 : => inf 값 나와서 실패.. 왜지2... ^

submission(pipe_voting)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: RuntimeWarning: overflow encountered in exp
  after removing the cwd from sys.path.


,datetime,count
0,2011-01-20 00:00:00,12.902914
1,2011-01-20 01:00:00,5.617978
2,2011-01-20 02:00:00,4.849945
3,2011-01-20 03:00:00,2.724142
4,2011-01-20 04:00:00,9.659242
...,...,...
6488,2012-12-31 19:00:00,0.000000
6489,2012-12-31 20:00:00,0.000000
6490,2012-12-31 21:00:00,0.000000
6491,2012-12-31 22:00:00,0.000000


[RMSLE 평가 지표에 대해](https://ahnjg.tistory.com/90)

요약 
1. 큰 것보다 적은 것을 오차없이 예측할 때 점수가 더 좋음.
2. under estimator에 대해 페널티를 부과한다. <b> 예측값 > 실제값</b> 보다 <b>예측값 < 실제값</b>일 때, 점수가 안 좋음.

## 파라미터 찾기

In [13]:
# grid_search - RandomForestRegressor^
param_grid={'n_estimators': range(100, 500, 100), 
            'max_features': ['auto', 'sqrt', 'log2']}
from sklearn.model_selection import GridSearchCV
grid_search=GridSearchCV(RandomForestRegressor(), param_grid, cv=5)
grid_search.fit(X_features, target)

print("Best Parameter: {}".format(grid_search.best_params_))
print("Best Score: {:.4f}".format(grid_search.best_score_))

# Best Parameter: {'max_features': 'auto', 'n_estimators': 400}
# Best Score: 0.8092 

Best Parameter: {'max_features': 'auto', 'n_estimators': 400}
Best Score: 0.8092


In [14]:
# Random_search - RandomForestRegressor^
from scipy.stats import randint
param_distribs = {'n_estimators': randint(low=100, high=500), 
                  'max_features': ['auto', 'sqrt', 'log2']}
from sklearn.model_selection import RandomizedSearchCV
random_search=RandomizedSearchCV(RandomForestRegressor(), 
                                 param_distributions=param_distribs, n_iter=20, cv=5)
random_search.fit(X_features, target)


print("Best Parameter: {}".format(random_search.best_params_))
print("Best Score: {:.4f}".format(random_search.best_score_))

# Best Parameter: {'max_features': 'auto', 'n_estimators': 365}
# Best Score: 0.8107

Best Parameter: {'max_features': 'auto', 'n_estimators': 365}
Best Score: 0.8107


In [19]:
def search_params(x, y, model, paras, n = 5, scorer = rmsle_scorer) :
    # pipeline 으로 받은 모델을 부르고
    model = model['model']

    # kfold 
    kf = KFold(n_splits = n, shuffle=True, random_state = 0)

    grid_model = GridSearchCV(estimator = model, param_grid = paras, cv=kf, n_jobs=-1, verbose=2, scoring = scorer)
    grid_model.fit(x,y)
    
    # grid_search한 결과를 Dataframe화
    scores_df = pd.DataFrame(grid_model.cv_results_)
    scores_df[['params', 'mean_test_score','rank_test_score', 'split0_test_score', 'split1_test_score','split2_test_score']]
    
    # GridSearchCV 의 best score는 높은 점수를 알려줌. 
    # 즉, 현재 우리의 평가지표(rslme)는 낮은 점수일수록 좋은 평가이기 때문에 score를 오름차순으로 정렬해서 확인
    scores_df = scores_df.sort_values('mean_test_score', ascending = True).loc[:,['params','mean_test_score']] 
    # 낮은 순으로 정렬: 가장 맨 위에 있는게 ,점수가 제일 좋은 점수가 올라오니까, 그게 베스트 파라미터다.

    return(scores_df)

In [20]:
#lgbm에 대한 parameters
para_lgbm = [{
    'learning_rate' : [0.01, 0.03, 0.05, 0.07, 0.1],
    'n_estimators' : [500, 800, 1000, 1300, 1500],
    'random_state' : [0]}]
    
search_params(X_features, target, pipe_lgbm, para_lgbm)

Fitting 5 folds for each of 25 candidates, totalling 125 fits


,params,mean_test_score
9,"{'learning_rate': 0.03, 'n_estimators': 1500, ...",0.286212
8,"{'learning_rate': 0.03, 'n_estimators': 1300, ...",0.286435
12,"{'learning_rate': 0.05, 'n_estimators': 1000, ...",0.286482
13,"{'learning_rate': 0.05, 'n_estimators': 1300, ...",0.286511
14,"{'learning_rate': 0.05, 'n_estimators': 1500, ...",0.286768
16,"{'learning_rate': 0.07, 'n_estimators': 800, '...",0.287034
11,"{'learning_rate': 0.05, 'n_estimators': 800, '...",0.287039
17,"{'learning_rate': 0.07, 'n_estimators': 1000, ...",0.287131
18,"{'learning_rate': 0.07, 'n_estimators': 1300, ...",0.288150
7,"{'learning_rate': 0.03, 'n_estimators': 1000, ...",0.288325


1000/0.03 이 best score인 것을 확인할 수 있음

In [17]:
# lgbm (1500, 0.03, random_state = 0)
pipe_lgbm4 = Pipeline([('model', LGBMRegressor(n_estimators=1500, learning_rate = 0.03))])

submission(pipe_lgbm4) # 0.399

,datetime,count
0,2011-01-20 00:00:00,11.755624
1,2011-01-20 01:00:00,5.278450
2,2011-01-20 02:00:00,5.531308
3,2011-01-20 03:00:00,3.034705
4,2011-01-20 04:00:00,2.813775
...,...,...
6488,2012-12-31 19:00:00,256.392373
6489,2012-12-31 20:00:00,150.115036
6490,2012-12-31 21:00:00,104.161593
6491,2012-12-31 22:00:00,86.741994


In [18]:
cv_lgbm5 = Pipeline([('model', LGBMRegressor(n_estimators=1500, learning_rate = 0.03, random_state=0))])
model5 = [cv_lgbm5]
cv_score(model5)     

#   파라미터 조정 후         파라미터 조정 전
# 0.28621  / 0.00548   =>    0.32172  / 0.0055

LGBMRegressor(learning_rate=0.03, n_estimators=1500, random_state=0) 의 평균 score: 0.28621
LGBMRegressor(learning_rate=0.03, n_estimators=1500, random_state=0) 의 std: 0.00548

